In [1]:
import numpy as np
import pandas as pd
import cv2
from keras.applications.efficientnet_v2 import EfficientNetV2L
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from tqdm import tqdm

In [2]:
DATA_DIR = "../data/dog-breed-identification"

# csv 데이터 셋팅
df_train = pd.read_csv(DATA_DIR+"/labels.csv")
df_test = pd.read_csv(DATA_DIR+"/predict.csv")

# breed 파싱(중첩제거?)
targets_series = pd.Series(df_train['breed'])
# one_hot 변환(0 0 1 0)
one_hot = pd.get_dummies(targets_series, sparse = True)

# 배열로 변환
one_hot_labels = np.asarray(one_hot)

# 이미지 사이즈 정의
im_size = 90

y_train = []
x_test = []

# 예측할 분류 셋팅
i = 0
for f, breed in tqdm(df_train.values):
    label = one_hot_labels[i]
    y_train.append(label)
    i += 1

# 예측할 데이터 셋팅
cnt = 0
for f in tqdm(df_test['id'].values):
    img = cv2.imread(DATA_DIR+"/test/{}.jpg".format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))
    cnt+=1
    if(cnt == 1):
        break

y_train_raw = np.array(y_train, np.uint8)
x_test = np.array(x_test, np.float32) / 255.


# shape 확인
print(y_train_raw.shape)
print(x_test.shape)

# 분류
num_class = y_train_raw.shape[1]

  0%|                                                                                        | 0/10357 [00:00<?, ?it/s]

(10222, 120)
(1, 90, 90, 3)


In [4]:
# 사전 교육된 기본 모델 생성
base_model = EfficientNetV2L(
    # weights = 'imagenet',
    weights = None,
    include_top=False, input_shape=(im_size, im_size, 3))

# model out 정의
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 저장된 weights 로드
model.load_weights('EfficientNetV2L_DOG_weights.h5')
# model.summary()

Metal device set to: Apple M1 Pro


2022-09-19 03:48:34.654068: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-19 03:48:34.654185: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
import datetime
import tensorflow as tf

with tf.device('/CPU:0'):
    print("cpu predict start  :", datetime.datetime.now())
    # 예측할 데이터 결과
    preds = model.predict(x_test, verbose=1)
    print("cpu predict finish :", datetime.datetime.now())

sub = pd.DataFrame(preds)
# 열 이름을 앞서 한번의 키 인코딩으로 생성된 열 이름으로 설정
col_names = one_hot.columns.values
sub.columns = col_names

sub.head(5)

cpu predict start  : 2022-09-19 03:49:06.677321
1/1 [==============================] - 0s 213ms/step
cpu predict finish : 2022-09-19 03:49:06.920687


,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0.015936,0.000044,0.00007,0.008345,0.003395,0.00008,0.023703,0.012459,0.000115,0.001113,...,0.000083,0.000129,0.000006,0.016458,0.000091,0.000016,0.000409,0.000161,0.00002,0.004808


In [20]:
import datetime

print("gpu predict start  :", datetime.datetime.now())
# 예측할 데이터 결과
preds = model.predict(x_test, verbose=1)
print("gpu predict finish :", datetime.datetime.now())

sub = pd.DataFrame(preds)
# 열 이름을 앞서 한번의 키 인코딩으로 생성된 열 이름으로 설정
col_names = one_hot.columns.values
sub.columns = col_names

sub.head(5)

gpu predict start  : 2022-09-19 03:49:07.017077
1/1 [==============================] - 0s 76ms/step
gpu predict finish : 2022-09-19 03:49:07.117237


,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0.015936,0.000044,0.00007,0.008345,0.003395,0.00008,0.023704,0.012459,0.000115,0.001113,...,0.000083,0.000129,0.000006,0.016458,0.000091,0.000016,0.000409,0.000161,0.00002,0.004808
